Contains the complete step-wise filter, pull, and SonarQube analysis pipeline for the provided git repositories.  
The expected input is a list of repositories of which the data is acquired through the github web API.

Make sure that a SonarQube instance is running prior to running this script and that the credentials used in the analysis phase are set correctly.
Consider that executing this script takes quite a while to complete.

- SonarQube logs are put in the `./output` folder, you can use this to figure out why SonarQube fails when analyzing certain projects.
- The results are put in the `./results` folder.

If you interrupt this script halfway through, or the script crashes at some point, it might be that it doesn't work the next run.
Restarting Jupyter might help here, but also check if the access tokens are revoked correctly.
Additionally, at some point the SonarQube might run out of heap memory, which is why this script is executed in batches, each of which has a corresponding sonarqube instance.

This script uses bash calls to run SonarQube, so if you run this on Windows, these commands probably have to be changed somewhat.
It's definitely not perfect either; it is quite likely it will crash a few times whilst running it.


In [7]:
import array
import os
import requests
from git import Repo, GitCommandError
import subprocess
from typing import Tuple
import shutil
from subprocess import Popen
import time

with open("./data/all_repositories_batch_2.csv", "r") as data_file:
    data = [entry.strip().split(",") for entry in data_file.readlines()[1:]]

print(f'loaded data:\n{data}')


loaded data:
[['hawq', 'https://github.com/apache/hawq', '182943', 'master', 'C'], ['madlib', 'https://github.com/apache/madlib', '20065', 'master', 'C++'], ['incubator-heron', 'https://github.com/apache/incubator-heron', '620042', 'master', 'Java'], ['rya', 'https://github.com/apache/rya', '8748', 'master', 'Java'], ['mynewt-core', 'https://github.com/apache/mynewt-core', '180738', 'master', 'C'], ['systemds', 'https://github.com/apache/systemds', '333609', 'main', 'Java'], ['brooklyn-server', 'https://github.com/apache/brooklyn-server', '43347', 'master', 'Java'], ['fineract', 'https://github.com/apache/fineract', '52749', 'develop', 'Java'], ['lucene-solr', 'https://github.com/apache/lucene-solr', '585082', 'master', ''], ['beam', 'https://github.com/apache/beam', '267628', 'master', 'Java'], ['arrow', 'https://github.com/apache/arrow', '105924', 'master', 'C++'], ['incubator-retired-edgent', 'https://github.com/apache/incubator-retired-edgent', '25183', 'develop', 'Java'], ['nifi-m

Step 1 of the lifecycle, all methods used to statically filter the repositories.


In [8]:
PROJECT_TYPE = 4


def is_not_java(entry: array) -> bool:
    return entry[PROJECT_TYPE] != "Java"


def is_considered(entry: array) -> bool:
    """Returns true if the project is a maven project"""

    # return is_java(entry) and is_maven(entry)
    return is_not_java(entry)


Step 2 of the lifecycle, cloning a repository


In [9]:
PROJECT_NAME = 0
PROJECT_REPO_URL = 1
DEFAULT_BRANCH = 3

date = "May 7, 2020"


def clone_repository(entry: array) -> Tuple[int, str]:
    """
    Clones repository to the ./repos folder and 
    returns the status code and repo's folder.
    """

    url = entry[PROJECT_REPO_URL]
    dir = os.path.join("./repos", entry[PROJECT_NAME])

    try:
        Repo.clone_from(url, dir)
        status = 0

    except GitCommandError as e:
        if e.status != 128:
            status = e.status
        else:
            status = 0
            print(f'repository for {entry[PROJECT_NAME]} is already cloned')

    finally:
        return status, dir


def checkout_date(dir: str, entry: array):
    """Checks out the latest version of the default branch at the given time."""

    os.chdir(dir)

    args = [f'git rev-list -1 --before="{date}" {entry[DEFAULT_BRANCH]}']
    with Popen(args, stdout=subprocess.PIPE, stderr=None, shell=True) as get_commit:
        commit = get_commit.stdout.read().decode('UTF-8')

    args = [f'git checkout {commit}']
    with Popen(args, stdout=subprocess.PIPE, stderr=None, shell=True) as do_checkout: 
        status = do_checkout.returncode

    os.chdir("../..")

    return status


Implements steps 3 to 5 of the lifecycle, the SonarQube steps.


In [10]:
server_url = "http://sonarqube:9000"
auth = ('admin', 'password')


def create_sonarqube_project(entry: array) -> int:
    """Creates SonarQube project if none exists yet"""

    name = entry[PROJECT_NAME]

    url = f"{server_url}/api/projects/create"
    data = {'name': name, 'project': name, 'visibility': 'public'}
    c_res = requests.post(url=url, data=data, auth=auth)

    return c_res.status_code


def perform_sonarqube_analysis(entry: array, dir: str) -> int:
    """Executes sonarqube analysis and sends it to the server"""

    name = entry[PROJECT_NAME]

    # All java files are ignored as they require compiling and
    # will cause the analysis to fail.
    args = [
        'sonar-scanner',
        '-Dsonar.sources=.',
        f'-Dsonar.projectKey={name}',
        f'-Dsonar.host.url={server_url}',
        f'-Dsonar.login={auth[0]}',
        f'-Dsonar.password={auth[1]}',
        '-Dsonar.coverage.exclusions=/**.java',
        '-Dsonar.test.exclusions=/**.java',
        '-Dsonar.exclusions=/**.java'
    ]

    with open(f"./output/{name}-sonarqube-output.log", "w") as output_file:
        os.chdir(dir)
        res = subprocess.run(args, stdout=output_file)

    # TODO: this shouldn't assume two layers.
    os.chdir("../..")

    return res.returncode


def export_sonarqube_issues(entry: array):
    """Exports the generated issues through the web API"""

    name = entry[PROJECT_NAME]

    url = f"{server_url}/api/issues/search"
    data = {'componentKeys': name}
    res = requests.get(url=url, data=data, auth=auth)

    with open(f"./results/issues-{name}.json", "w") as results_file:
        results_file.write(res.text)


Step 6 of the lifecycle, clean up methods


In [11]:
def delete_sonarqube_project(entry: array):
    """Deletes sonarqube project """

    name = entry[PROJECT_NAME]

    url = f'{server_url}/api/projects/delete'
    data = {'project': name}
    requests.post(url=url, data=data, auth=auth)


def delete_repository(entry, dir):
    """Deletes repository"""

    shutil.rmtree(dir)


Implements the pipeline lifecycle.

1. filtering repositories
2. cloning repositories
3. creating SonarQube project
4. analyzing repository
5. exporting results
6. (optional) deleting sonarqube project


In [12]:
def perform_lifecycle(entry: array):

    name = entry[PROJECT_NAME]

    # step 1: filtering
    if not is_considered(entry):
        print(f"filtered out {name}.")
        return

    # step 2: cloning repositories.
    print(f'retrieving repository of {name}')
    status, dir = clone_repository(entry)
    if status != 0:
        print(f'cloning repository failed for {name} with status {status}')
        return

    time.sleep(1)
    # step 2a: checking out at date.
    print(f'checking out at date for {name}')
    checkout_date(dir, entry)

    # step 3: creating sonarqube project
    print(f'creating project for {name}')
    status = create_sonarqube_project(entry)
    if status != 200:
        print(f'creating project failed for {name}')
        return

    # step 4: running SonarQube
    print(f'running sonarqube on {name}')
    status = perform_sonarqube_analysis(entry, dir)
    if status != 0:
        print(f'sonarqube analysis failed for {name} with status {status}')
    else:
        # step 5: extracting sonarqube data
        print(f'exporting results of {name}')
        export_sonarqube_issues(entry)

    # step 6: deleting sonarqube project
    print(f'cleaning up after {name}')
    # delete_sonarqube_project(entry)
    # delete_repository(entry, dir)

    print(f'completed analysis on {name}')


for entry in data:
    perform_lifecycle(entry)


retrieving repository of hawq
checking out at date for hawq


Note: switching to '67be67a872153b998d41c70b0bc0c6d358da28be'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 67be67a8 HAWQ-1764. fix udf debug core and improve timing accuracy


creating project for hawq
running sonarqube on hawq


ERROR: Unable to parse file: tools/bin/pythonSrc/pychecker-0.8.18/test_input/test8.py
ERROR: Parse error at line 4 column 11:

    1: "doc"
    2: 
    3: def func():
  -->      syntax error here
    5: EOF

ERROR: Unable to parse file: src/bin/gpupgrade/transform.py
ERROR: Parse error at line 115 column 4:

  109: (tfile),
  110:                                            'upg2_pg_class_todata.data']))
  111:     touch_files(os.path.join([path, 'base', dbname_oid(db)], new_files)
  112: 
  113: 
  114: 
  -->      cmd = ' '.join(['/Users/swm/greenplum-db-devel/bin/postgres', '--single', '-O', '-P',
  116:                     '-c', 'gp_session_role=utility',
  117:                     '-c', 'exit_on_error=true',
  118:                     '-D', path, db])

ERROR: Unable to parse file: tools/bin/pythonSrc/pychecker-0.8.18/pychecker2/tests/badparse.py
ERROR: Parse error at line 1 column 0:

  -->  ===
    2: EOF

ERROR: Unable to parse file: tools/bin/pythonSrc/pychecker-0.8.18/test_inpu

exporting results of hawq
cleaning up after hawq
completed analysis on hawq
retrieving repository of madlib
checking out at date for madlib
creating project for madlib


...

Note: switching to '8f05da6372f7498c5e170a29c816d654ae74adfa'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 8f05da6 SDAP-220: temporal variance algorithm (#93)


running sonarqube on incubator-sdap-nexus
exporting results of incubator-sdap-nexus
cleaning up after incubator-sdap-nexus
completed analysis on incubator-sdap-nexus
retrieving repository of openwhisk-release
checking out at date for openwhisk-release
creating project for openwhisk-release


Note: switching to '5f921bb804abd25898dc104ee681a030135b6143'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 5f921bb openwhisk-client-js v3.21.2 (#335)


running sonarqube on openwhisk-release
exporting results of openwhisk-release
cleaning up after openwhisk-release
completed analysis on openwhisk-release
filtered out plc4x.
retrieving repository of skywalking-website


UnboundLocalError: local variable 'status' referenced before assignment